# PCA  for Feature Selection

## Purpose: 

* Load Normalized Matrices 
* Scale Matrices
* PCA 
* Save Top "x" Eigenvalues
* Output Values

## Packages

### Install "scikit-learn" 1.0.0

In [1]:
! pip3 install --user scikit-learn==1.0.0

In [2]:
import sys, importlib
sys.path.insert(0,"/home/yraghav/.local/lib/python3.9/site-packages/scikit_learn.libs")

In [3]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sklearn
from sklearn.decomposition import PCA, SparsePCA
from sklearn.preprocessing import StandardScaler

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-sr_9jkfu because the default path (/home/jovyan/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [4]:
sklearn.__version__ == "1.0"

True

## Load Data

In [5]:
all_matrices = {}
scaled_matrices={}

PCA_objects = {}
eigenvalues = {}

selected_features = {}

In [6]:
files = glob.glob("/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/1_feature_generation_and_pre_processing/raw_counts_processing/output/*.normalized.matrix")
assert len(files)==5

In [7]:

for file in files: 
    # Take transpose of matrix 
    matrix = pd.read_csv(file, sep="\t", header=0).T
    # make first row as header
    matrix.columns = matrix.iloc[0]
    matrix = matrix[1:]
    
    matrix
    
    # append matrix by assay name 
    all_matrices[file.split("/")[-1].split(".")[0]] = matrix

Index,chr1:56120-56475,chr1:66798-67060,chr1:180739-181148,chr1:629082-630053,chr1:630221-630503,chr1:630717-631417,chr1:631809-633007,chr1:633382-633554,chr1:633880-634642,chr1:634684-634906,...,chrY:7287610-7288291,chrY:9141824-9142031,chrY:11329794-11330046,chrY:11789193-11789460,chrY:12438835-12439262,chrY:12446469-12446811,chrY:12448053-12448296,chrY:12450934-12451269,chrY:24270746-24270988,chrY:25399151-25399399
'201209Fra_D20-6795_1.trim.nodup.no_chrM_MT',10.452051,17.420085,25.259124,56043.898615,10356.240746,35161.571292,59077.60648,6606.567372,33646.89487,5478.616845,...,42.679209,264.785297,20.033098,12.19406,27.872137,20.033098,20.904102,15.678077,10.452051,13.936068
'201209FraA_D20-6801_1.trim.nodup.no_chrM_MT',13.398295,23.142509,21.924482,97655.298417,18147.381242,63127.892814,105574.908648,11098.660171,59450.669915,9710.109622,...,52.375152,526.187576,17.052375,38.976857,85.261876,41.412911,29.232643,51.157125,30.45067,9.744214
'72-20_rep3_210203Fra_D21-1102_1.trim.merged.nodup.no_chrM_MT',57.940479,23.702923,51.356334,50903.344746,13540.953384,42230.708454,66099.552278,7054.253358,42141.164077,6661.838293,...,100.07901,808.533052,47.405847,39.504872,79.009745,92.178035,23.702923,118.514617,51.356334,57.940479
'201209FraB_D20-7056_1.trim.nodup.no_chrM_MT',11.367611,11.367611,17.488632,63245.890171,12226.302903,39937.040923,68470.619098,7701.993704,39061.734872,6211.962225,...,33.228402,234.347674,14.865338,25.358517,38.474991,14.865338,20.986359,20.111927,10.493179,6.121021
'56-22_rep3_210203Fra_D21-1101_1.trim.merged.nodup.no_chrM_MT',13.992921,26.339616,25.516503,55594.699152,10502.922051,33210.963865,59720.964688,6405.46547,33115.482756,5430.07655,...,35.39386,255.165034,13.169808,8.23113,41.978764,9.054243,15.639147,25.516503,9.054243,7.408017
'201209FraB_D20-6811_1.trim.nodup.no_chrM_MT',24.659696,18.741369,19.727757,73510.55435,13831.1304,47423.554942,81045.571119,8733.478004,45407.378181,7629.710002,...,53.264944,426.11955,32.550799,7.891103,48.333005,23.673308,27.61886,26.632472,14.795818,12.823042
'201209FraA_D20-6802_1.trim.nodup.no_chrM_MT',16.166282,18.475751,30.023095,91517.321016,17314.08776,59259.815242,99881.062356,10506.928406,55193.995381,9101.616628,...,75.057737,551.963048,16.166282,24.249423,53.117783,43.879908,24.249423,27.713626,31.177829,36.951501
'201209FraB_D20-6808_1.trim.nodup.no_chrM_MT',10.253255,16.405209,21.531836,73205.167641,14051.061212,46893.263611,79549.882088,8790.115862,44839.536553,7341.330873,...,37.937045,319.901569,17.430534,14.354558,46.139649,21.531836,18.45586,32.810417,11.278581,14.354558
'201209FraA_D20-6803_1.trim.nodup.no_chrM_MT',8.558979,13.227513,14.783691,44541.705571,8583.877996,28669.467787,48145.035792,5037.348273,27135.854342,4663.087457,...,28.789293,259.88173,14.783691,12.449424,31.123561,22.564581,14.005602,26.455026,7.78089,7.002801
'201209FraB_D20-6809_1.trim.nodup.no_chrM_MT',14.861996,16.985138,16.985138,68242.038217,12900.212314,43752.653928,74445.859873,8152.866242,41918.259023,6818.471338,...,66.878981,301.4862,19.10828,23.354565,39.278132,42.462845,14.861996,39.278132,19.10828,12.738854


Index,chr1:777598-778094,chr1:812028-812445,chr1:843880-846421,chr1:846525-846980,chr1:858133-859349,chr1:904819-905892,chr1:920443-921735,chr1:922979-926655,chr1:934940-936273,chr1:938851-943841,...,chrX:155677914-155678902,chrX:155679246-155679623,chrX:155728452-155728764,chrX:155733538-155734427,chrX:155765321-155767187,chrX:155767962-155771723,chrX:155819525-155820214,chrX:155887996-155888283,chrX:155956864-155958168,chrX:155958979-155960590
'20-20_H3K4me1_rep2_210115Fra_D20-3774-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',42.310822,47.19284,353.132628,42.310822,104.149715,84.621644,175.752644,333.604557,128.559805,650.93572,...,104.149715,19.528072,17.900732,87.876322,222.945484,476.810415,81.366965,32.546786,162.73393,172.497966
'22-20_H3K4me1_rep1_210115Fra_D20-5403-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',28.633761,25.906736,272.702482,24.543223,77.720207,59.994546,152.71339,293.155168,160.894464,508.590128,...,66.812108,31.360785,16.362149,65.448596,162.257977,319.061903,61.358058,9.544587,89.991819,122.716117
'22-20_H3K4me1_rep3-1_210115Fra_D20-4761-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',19.634311,26.997178,231.930298,25.770033,56.448644,73.628666,90.808688,279.788931,120.260155,458.952019,...,41.722911,14.725733,20.861455,66.265799,147.257332,261.381765,36.814333,11.0443,98.171555,121.487299
'56-22_H3K4me1_rep1_210115Fra_D20-4168-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',45.468324,40.921491,321.309488,34.859048,83.358594,96.999091,175.810852,401.63686,203.091846,656.259473,...,104.577145,28.796605,24.249773,84.874204,195.513792,409.214914,66.686875,27.280994,140.951804,172.77963
'56-22_H3K4me1_rep1-1_210115Fra_D20-5523-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',33.628319,53.097345,430.088496,40.707965,113.274336,141.59292,265.486726,484.955752,330.973451,872.566372,...,109.734513,30.088496,47.787611,159.292035,240.707965,523.893805,69.026549,35.39823,173.451327,235.39823
'56-22_H3K4me1_rep3_210115Fra_D20-3539-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',42.053364,36.2529,317.575406,43.50348,98.607889,65.25522,149.361949,327.726218,126.160093,504.640371,...,76.856148,20.301624,18.851508,76.856148,178.364269,387.180974,49.303944,24.651972,124.709977,181.264501
'72-20_H3K4me1_rep3_210115Fra_D20-3542-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',20.457867,15.586946,150.998539,14.612762,39.941549,24.354603,56.502679,150.024355,57.476863,227.959084,...,28.25134,6.819289,3.896736,32.148076,65.270336,184.120799,20.457867,11.690209,53.580127,77.93473
'22-20_H3K4me1_rep2_210115Fra_D20-3771-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',14.991005,15.990406,150.909454,12.992205,54.96702,33.979612,74.955027,154.907056,67.959224,240.855487,...,19.988007,10.993404,6.995803,31.980812,88.946632,145.912453,13.991605,5.996402,56.965821,76.953828
'72-20_H3K4me1_rep1_210115Fra_D20-4025-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',24.564538,21.214828,176.418044,27.914247,49.129075,51.362215,94.908441,224.430549,98.258151,399.732023,...,33.497097,14.515409,12.282269,41.313086,102.724431,228.896829,36.846807,7.815989,87.092452,98.258151
'22-20_H3K4me1_rep3_210115Fra_D20-3335-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',9.832842,14.749263,129.79351,15.732547,41.297935,23.59882,64.896755,127.826942,65.880039,263.520157,...,34.414946,9.832842,7.866273,30.481809,81.612586,178.957719,17.699115,11.79941,58.99705,83.579154


Index,chr1:777447-779497,chr1:779576-780466,chr1:825910-827899,chr1:828175-828433,chr1:844631-844974,chr1:845292-846240,chr1:858316-859317,chr1:869626-870334,chr1:903738-905853,chr1:920613-921549,...,chrX:155641083-155641852,chrX:155641923-155642556,chrX:155766343-155771075,chrX:155880248-155882605,chrX:155957211-155958285,chrX:155958666-155960245,chrX:156016392-156016788,chrY:5003106-5003427,chrY:12564838-12565123,chrY:15513814-15514066
'20-20_H3K4me3_rep3_210115Fra_D20-4765-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',165.330661,27.55511,154.058116,6.888778,1.878758,16.282565,10.646293,16.282565,139.028056,16.282565,...,37.57515,11.898798,264.90481,170.966934,23.797595,90.180361,0.0,8.767535,4.383768,2.50501
'22-20_H3K4me3_rep3_210115Fra_D20-4762-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',220.564385,39.734025,220.564385,1.621797,8.108985,19.461563,30.003244,32.435939,196.237431,34.868634,...,53.519299,4.865391,347.064548,299.221537,18.650665,88.387934,9.730782,15.407071,6.487188,4.054492
'72-20_H3K4me3_rep1_210115Fra_D20-5527-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',411.913054,74.456587,377.086586,8.406389,10.808214,33.625555,37.228294,56.442897,366.278372,80.46115,...,110.483968,27.620992,698.931188,510.387895,61.246547,259.397142,22.817341,26.420079,13.21004,14.410952
'72-20_H3K4me3_rep3_210115Fra_D20-3541-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',329.196466,59.949516,297.644089,13.672697,18.931426,35.759361,35.759361,28.397139,263.98822,45.225074,...,73.622213,22.086664,560.580564,396.508204,23.13841,170.382836,11.569205,21.034918,7.362221,8.413967
'20-20_H3K4me3_rep1_210115Fra_D20-5407-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',570.799458,106.707317,552.168022,18.631436,35.569106,99.932249,88.075881,108.401084,601.287263,123.644986,...,176.151762,38.95664,1068.766938,782.520325,120.257453,411.585366,0.0,40.650407,11.856369,11.856369
'22-20_H3K4me3_rep1_210115Fra_D20-5404-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',263.386397,44.380125,275.928606,12.542209,16.401351,27.978775,49.204052,34.732272,215.14713,51.133623,...,59.816691,18.330921,446.69561,364.688857,21.225277,110.950314,11.577424,14.47178,8.683068,5.788712
'56-22_H3K4me3_rep3_210115Fra_D20-3540-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',372.329173,94.140047,320.49926,8.462027,15.8663,37.021367,29.617093,34.90586,253.8608,49.714407,...,85.67802,44.42564,581.764333,419.928073,25.38608,199.91538,22.21282,19.03956,9.51978,11.635287
'56-22_H3K4me3_rep1_210115Fra_D20-5524-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',394.230769,97.355769,387.019231,14.423077,9.615385,44.471154,62.5,64.903846,371.394231,82.932692,...,112.980769,32.451923,725.961538,557.692308,28.846154,221.153846,22.836538,26.442308,13.221154,10.817308
'72-20_H3K4me3_rep2_210115Fra_D20-5201-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',424.320828,72.445019,439.844761,12.936611,14.230272,55.627426,51.746442,65.976714,355.756792,80.206986,...,144.890039,41.397154,815.006468,624.838292,62.095731,280.72445,29.754204,29.754204,14.230272,15.523933
'56-22_H3K4me3_rep2_210115Fra_D20-3778-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',219.396852,52.717265,186.060935,7.752539,3.101016,13.95457,17.83084,22.482363,180.634158,30.234902,...,44.964726,13.95457,297.697496,231.800915,16.280332,90.704706,7.752539,11.628808,6.202031,7.752539


Index,chr1:778236-779421,chr1:826712-827854,chr1:958786-959213,chr1:1012976-1013414,chr1:1013515-1013891,chr1:1019478-1020531,chr1:1024870-1025616,chr1:1069151-1069387,chr1:1115718-1116122,chr1:1144228-1144907,...,chrX:155071455-155072119,chrX:155215871-155216250,chrX:155216482-155217124,chrX:155263832-155264576,chrX:155612278-155613254,chrX:155641392-155641820,chrX:155770022-155770479,chrX:155881365-155881768,chrX:155957874-155958216,chrX:155958842-155959382
'20-20_H3K27Ac_rep2_210115Fra_D20-3776-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',41.869606,11.108263,5.981372,7.690336,5.981372,32.470307,26.488934,0.854482,7.690336,19.65308,...,25.634453,5.981372,23.925489,8.544818,20.507562,13.671708,7.690336,5.981372,5.981372,21.362044
'72-20_H3K27Ac_rep3_210115Fra_D20-4895-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',35.613593,20.032646,11.871198,5.935599,3.709749,42.291141,24.484345,8.161448,17.064846,14.838997,...,24.484345,3.709749,26.710194,11.871198,15.580947,11.871198,14.838997,6.677549,5.193649,14.097047
'22-20_H3K27Ac_rep2_210115Fra_D20-3773-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',21.019799,10.848929,5.424464,2.712232,3.39029,21.697857,22.375915,2.034174,6.102522,10.848929,...,7.458638,5.424464,12.883103,4.746406,9.492813,8.136697,5.424464,2.712232,4.068348,9.492813
'72-20_H3K27Ac_rep1_210115Fra_D20-5526-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',117.346346,69.710701,24.398745,23.2369,20.91321,81.329151,62.739631,19.751365,45.311955,39.50273,...,62.739631,18.58952,72.034391,56.930405,74.358081,39.50273,19.751365,29.046125,11.61845,40.664575
'20-20_H3K27Ac_rep2-1_210115Fra_D20-5076-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',163.804071,97.010178,30.216285,34.987277,38.167939,109.732824,97.010178,23.854962,57.251908,50.890585,...,114.503817,42.938931,122.455471,60.43257,120.86514,57.251908,41.348601,44.529262,33.396947,69.974555
'56-22_H3K27Ac_rep1_210115Fra_D20-5522-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',142.938822,71.469411,38.593482,45.740423,45.740423,107.204117,117.209834,25.728988,55.746141,44.311035,...,82.904517,17.152659,92.910234,44.311035,88.62207,41.452258,42.881647,41.452258,18.582047,52.887364
'22-20_H3K27Ac_rep1_210115Fra_D20-5402-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',126.457777,112.406913,32.316987,40.747506,47.772938,127.862864,105.381481,39.34242,74.46958,64.633975,...,91.330617,29.506815,113.811999,96.950962,108.191654,37.937333,37.937333,57.608543,15.455951,57.608543
'22-20_H3K27Ac_rep3_210115Fra_D20-4763-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',49.075454,33.301201,14.021558,18.403295,7.887126,41.188327,35.930243,7.887126,21.908685,15.774253,...,32.424853,10.516169,35.053895,14.897906,21.908685,13.145211,11.392516,14.897906,8.763474,15.774253
'20-20_H3K27Ac_rep1_210115Fra_D20-5406-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',182.463254,133.468491,60.821085,42.236864,52.373712,140.22639,148.673762,40.54739,62.510559,79.405305,...,140.22639,50.684237,148.673762,94.610576,135.157966,69.268458,42.236864,67.578983,33.789491,87.852678
'20-20_H3K27Ac_rep3_210115Fra_D20-3538-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',51.116211,17.734196,8.345504,17.734196,16.691008,40.684331,46.943459,7.302316,10.43188,28.166076,...,25.036512,8.345504,46.943459,20.86376,29.209264,18.777384,12.518256,8.345504,15.64782,23.993324


Index,chr1:788527-790364,chr1:860313-861171,chr1:861641-864435,chr1:864574-865030,chr1:865151-866637,chr1:866891-872775,chr1:872989-873634,chr1:874109-875862,chr1:882342-891444,chr1:891658-892553,...,chrY:56859503-56860565,chrY:56860859-56861167,chrY:56863278-56863932,chrY:56866321-56866861,chrY:56867506-56870396,chrY:56870614-56871622,chrY:56871910-56872550,chrY:56872818-56873204,chrY:56873557-56874690,chrY:56879321-56879994
'56-22_H3K27me3_rep3_210115Fra_D20-4893-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',10.904091,3.792727,17.541364,1.896364,8.533637,62.580003,3.318637,14.222728,59.261366,3.318637,...,8.533637,0.948182,3.792727,4.266818,31.290001,10.43,5.689091,4.266818,13.274546,6.163182
'20-20_H3K27me3_rep3-1_210115Fra_D20-4766-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',25.5809,22.383287,69.281603,14.922191,45.832445,435.941164,10.658708,49.030058,370.923044,24.515029,...,18.119804,3.197612,7.461096,8.526967,81.006182,42.634833,10.658708,12.79045,42.634833,18.119804
'20-20_H3K27me3_rep2_210115Fra_D20-5077-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',28.738808,30.949486,107.217862,14.369404,48.634907,422.239416,24.317453,56.372278,378.025865,19.896098,...,26.528131,5.526694,13.264065,9.948049,68.531005,28.738808,9.948049,11.053388,39.792196,15.474743
'22-20_H3K27me3_rep3_210115Fra_D20-4764-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',23.966037,6.162695,29.443988,6.847439,14.379622,159.54533,3.42372,13.694878,119.14544,4.793207,...,28.759244,5.477951,10.271159,10.955902,56.149,30.128732,9.586415,10.271159,26.020268,11.640646
'22-20_H3K27me3_rep1_210115Fra_D20-4169-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',54.360671,35.452612,88.631529,9.45403,49.633656,176.081305,28.362089,75.632238,212.71567,18.90806,...,29.543843,0.0,20.089813,25.998582,157.173245,88.631529,44.906641,23.635074,81.541007,34.270858
'56-22_H3K27me3_rep2_210115Fra_D20-5079-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',41.418862,21.240442,101.954121,7.434155,40.356839,363.211555,15.930331,47.790994,368.521665,14.868309,...,27.612574,4.248088,8.496177,9.558199,63.721325,33.984707,11.682243,3.186066,37.170773,15.930331
'56-22_H3K27me3_rep1_210115Fra_D20-5525-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',46.739626,23.939808,113.999088,11.399909,49.019608,424.076607,14.819881,63.839489,375.057,23.939808,...,41.039672,7.979936,21.659827,17.099863,107.159143,58.139535,28.499772,14.819881,63.839489,19.379845
'72-20_H3K27me3_rep3_210115Fra_D20-4896-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',18.961404,4.281607,25.077986,3.058291,11.009848,118.661692,4.281607,17.12643,88.69044,3.669949,...,7.951557,2.446633,5.504924,9.174873,47.70934,26.912961,7.951557,7.339898,19.573063,6.72824
'72-20_H3K27me3_rep1_210115Fra_D20-5528-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',36.565547,47.399783,169.28494,20.314193,73.131094,637.865655,18.959913,83.96533,595.88299,40.628386,...,39.274106,8.125677,17.605634,17.605634,124.593716,62.296858,29.79415,25.731311,77.193933,21.668472
'22-20_H3K27me3_rep1-1_210115Fra_D20-5405-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',41.884817,8.376963,80.628272,4.188482,41.884817,340.314136,14.659686,47.120419,310.994764,17.801047,...,33.507853,9.424084,16.753927,14.659686,85.863874,45.026178,23.036649,12.565445,48.167539,19.895288


## Scale Data

In [8]:
for assay in all_matrices: 
    scaled_array = StandardScaler().fit_transform(all_matrices[assay])
    scaled_matrices[assay] = pd.DataFrame(scaled_array, columns = all_matrices[assay].columns, index = all_matrices[assay].index)
    
    scaled_matrices[assay]

Index,chr1:56120-56475,chr1:66798-67060,chr1:180739-181148,chr1:629082-630053,chr1:630221-630503,chr1:630717-631417,chr1:631809-633007,chr1:633382-633554,chr1:633880-634642,chr1:634684-634906,...,chrY:7287610-7288291,chrY:9141824-9142031,chrY:11329794-11330046,chrY:11789193-11789460,chrY:12438835-12439262,chrY:12446469-12446811,chrY:12448053-12448296,chrY:12450934-12451269,chrY:24270746-24270988,chrY:25399151-25399399
'201209Fra_D20-6795_1.trim.nodup.no_chrM_MT',-0.605266,-0.445489,-0.063346,-0.767547,-0.935222,-0.853251,-0.894574,-0.820746,-0.919508,-0.911202,...,-0.549316,-0.829336,-0.259734,-0.757878,-1.154099,-0.561122,-0.312390,-0.921940,-0.713432,-0.291057
'201209FraA_D20-6801_1.trim.nodup.no_chrM_MT',-0.445602,0.158073,-0.402275,0.987382,0.904665,1.000963,0.974129,0.901357,1.006818,0.950689,...,-0.051649,0.523312,-0.532053,1.419040,1.523013,0.324740,0.478105,0.370541,0.681924,-0.577340
'72-20_rep3_210203Fra_D21-1102_1.trim.merged.nodup.no_chrM_MT',1.968245,0.217182,2.589143,-0.984346,-0.183148,-0.384556,-0.612366,-0.649120,-0.285386,-0.390575,...,2.396861,1.984332,2.241035,1.461957,1.231364,2.428167,-0.046743,2.824336,2.140566,2.714233
'201209FraB_D20-7056_1.trim.nodup.no_chrM_MT',-0.555650,-1.083862,-0.853129,-0.463809,-0.493604,-0.536630,-0.517074,-0.400800,-0.515275,-0.588524,...,-1.034400,-0.986838,-0.731860,0.312135,-0.659498,-0.775245,-0.304583,-0.760417,-0.710562,-0.824787
'56-22_rep3_210203Fra_D21-1101_1.trim.merged.nodup.no_chrM_MT',-0.413378,0.495282,-0.037186,-0.786492,-0.900583,-0.982580,-0.868718,-0.897841,-0.959179,-0.932560,...,-0.923253,-0.879117,-0.886763,-1.079986,-0.496054,-1.016025,-0.812109,-0.563532,-0.810961,-0.736891
'201209FraB_D20-6811_1.trim.nodup.no_chrM_MT',0.164680,-0.306129,-0.625547,-0.030904,-0.114622,-0.040261,-0.011693,-0.005367,-0.041554,0.035296,...,-0.005979,0.005501,0.883881,-1.107624,-0.199642,-0.410292,0.324935,-0.522878,-0.410356,-0.367071
'201209FraA_D20-6802_1.trim.nodup.no_chrM_MT',-0.295598,-0.334145,0.420858,0.728517,0.707882,0.744503,0.745296,0.674508,0.689045,0.682947,...,1.112587,0.656689,-0.613006,0.221987,0.023558,0.426958,0.005128,-0.483492,0.732660,1.280787
'201209FraB_D20-6808_1.trim.nodup.no_chrM_MT',-0.616039,-0.552531,-0.442183,-0.043784,-0.062685,-0.075420,-0.071804,0.016346,-0.083945,-0.091593,...,-0.792718,-0.544133,-0.497504,-0.582271,-0.301957,-0.499022,-0.544763,-0.297819,-0.655763,-0.262476
'201209FraA_D20-6803_1.trim.nodup.no_chrM_MT',-0.707856,-0.887693,-1.128056,-1.252644,-1.353767,-1.283689,-1.333949,-1.422327,-1.405576,-1.270042,...,-1.262248,-0.854710,-0.739319,-0.737121,-1.002427,-0.456231,-0.967155,-0.529342,-0.899806,-0.764565
'201209FraB_D20-6809_1.trim.nodup.no_chrM_MT',-0.366281,-0.491364,-0.904304,-0.253100,-0.334460,-0.283648,-0.276932,-0.227952,-0.302027,-0.321655,...,0.692793,-0.639424,-0.344226,0.149253,-0.622033,0.368243,-0.885871,-0.062204,-0.109464,-0.372821


Index,chr1:777598-778094,chr1:812028-812445,chr1:843880-846421,chr1:846525-846980,chr1:858133-859349,chr1:904819-905892,chr1:920443-921735,chr1:922979-926655,chr1:934940-936273,chr1:938851-943841,...,chrX:155677914-155678902,chrX:155679246-155679623,chrX:155728452-155728764,chrX:155733538-155734427,chrX:155765321-155767187,chrX:155767962-155771723,chrX:155819525-155820214,chrX:155887996-155888283,chrX:155956864-155958168,chrX:155958979-155960590
'20-20_H3K4me1_rep2_210115Fra_D20-3774-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.443697,1.513761,1.270638,1.441007,1.401571,0.874336,1.038974,0.838940,0.234034,1.156937,...,1.546760,0.474018,0.296784,0.866503,1.448261,1.495891,1.854856,1.647199,1.552301,1.010923
'22-20_H3K4me1_rep1_210115Fra_D20-5403-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.405626,0.096125,0.582101,0.116808,0.583820,0.187380,0.692097,0.526066,0.643958,0.514674,...,0.497880,1.707381,0.163365,0.301596,0.616099,0.466844,1.034779,-0.471754,0.125083,0.241711
'22-20_H3K4me1_rep3-1_210115Fra_D20-4761-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.277420,0.168747,0.233064,0.208241,-0.074340,0.567694,-0.239934,0.422678,0.128814,0.290707,...,-0.206921,-0.026546,0.553525,0.322179,0.410407,0.090576,0.028840,-0.333601,0.285572,0.222724
'56-22_H3K4me1_rep1_210115Fra_D20-4168-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.683347,1.096094,0.998210,0.885635,0.758276,1.219597,1.039851,1.365167,1.178918,1.180958,...,1.558767,1.440108,0.847346,0.790886,1.072111,1.054943,1.253184,1.162116,1.124931,1.015275
'56-22_H3K4me1_rep1-1_210115Fra_D20-5523-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.784706,1.906995,1.929433,1.321548,1.683894,2.463513,2.390003,2.009635,2.800145,2.156933,...,1.703647,1.574766,2.888445,2.665313,1.691825,1.803032,1.349077,1.909873,1.762579,1.982835
'56-22_H3K4me1_rep3_210115Fra_D20-3539-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.424156,0.785170,0.966244,1.529894,1.230102,0.334123,0.641638,0.793471,0.203611,0.496853,...,0.780035,0.554648,0.379232,0.588928,0.836953,0.911208,0.540734,0.919932,0.806262,1.146380
'72-20_H3K4me1_rep3_210115Fra_D20-3542-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.214913,-0.591164,-0.459767,-0.623297,-0.585084,-0.806774,-0.756442,-0.581045,-0.667125,-0.751532,...,-0.585362,-0.850661,-0.917581,-0.537174,-0.713819,-0.413424,-0.641539,-0.274100,-0.589323,-0.450235
'22-20_H3K4me1_rep2_210115Fra_D20-3771-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.629840,-0.564294,-0.460530,-0.744076,-0.120183,-0.538290,-0.478626,-0.543278,-0.534234,-0.693343,...,-0.817493,-0.415579,-0.648844,-0.541387,-0.389164,-0.662670,-0.906562,-0.798611,-0.522895,-0.465391
'72-20_H3K4me1_rep1_210115Fra_D20-4025-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.096778,-0.216352,-0.242159,0.368047,-0.300814,-0.053414,-0.178209,-0.005516,-0.150118,0.023507,...,-0.437999,-0.048469,-0.190424,-0.306326,-0.200240,-0.121334,0.030171,-0.630992,0.068197,-0.136205
'22-20_H3K4me1_rep3_210115Fra_D20-3335-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-1.021338,-0.646953,-0.641296,-0.539841,-0.543116,-0.827856,-0.630062,-0.752741,-0.560593,-0.591080,...,-0.412215,-0.536548,-0.573360,-0.579143,-0.489730,-0.447104,-0.754608,-0.264041,-0.483042,-0.363019


Index,chr1:777447-779497,chr1:779576-780466,chr1:825910-827899,chr1:828175-828433,chr1:844631-844974,chr1:845292-846240,chr1:858316-859317,chr1:869626-870334,chr1:903738-905853,chr1:920613-921549,...,chrX:155641083-155641852,chrX:155641923-155642556,chrX:155766343-155771075,chrX:155880248-155882605,chrX:155957211-155958285,chrX:155958666-155960245,chrX:156016392-156016788,chrY:5003106-5003427,chrY:12564838-12565123,chrY:15513814-15514066
'20-20_H3K4me3_rep3_210115Fra_D20-4765-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',-1.441506,-1.500643,-1.419126,-0.817603,-1.247291,-0.992567,-1.406355,-1.133109,-1.176670,-1.353430,...,-1.174208,-1.154977,-1.306714,-1.426471,-0.540477,-1.003528,-1.242227,-1.369746,-1.646949,-1.511622
'22-20_H3K4me3_rep3_210115Fra_D20-4762-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.946564,-1.004358,-0.816633,-2.007411,-0.523725,-0.848450,-0.455879,-0.477885,-0.708018,-0.715388,...,-0.787971,-1.752974,-0.946920,-0.675552,-0.713916,-1.022334,-0.242271,-0.596401,-0.952591,-1.128659
'72-20_H3K4me3_rep1_210115Fra_D20-5527-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.768085,0.410570,0.601331,-0.474776,-0.210242,-0.206339,-0.101110,0.495900,0.684934,0.849759,...,0.591965,0.181761,0.593974,0.560808,0.721456,0.771950,1.102531,0.686347,1.266684,1.430996
'72-20_H3K4me3_rep3_210115Fra_D20-3541-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.026873,-0.180586,-0.118353,0.714880,0.733171,-0.109605,-0.173238,-0.641710,-0.153013,-0.359861,...,-0.300989,-0.288780,-0.011891,-0.105947,-0.562690,-0.162017,-0.053351,0.059107,-0.663734,-0.051191
'20-20_H3K4me3_rep1_210115Fra_D20-5407-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',2.191844,1.724773,2.187427,1.835057,2.665438,2.799614,2.395639,2.603462,2.610094,2.332218,...,2.182728,1.145545,2.213558,2.154118,2.709972,2.368759,-1.242227,2.343835,0.819825,0.799617
'22-20_H3K4me3_rep1_210115Fra_D20-5404-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.562842,-0.815032,-0.315078,0.459504,0.439333,-0.462331,0.486931,-0.384740,-0.553113,-0.157027,...,-0.635420,-0.608102,-0.510616,-0.292247,-0.627158,-0.785603,-0.052507,-0.705340,-0.227710,-0.700038
'56-22_H3K4me3_rep3_210115Fra_D20-3540-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.413379,1.212662,0.088696,-0.462207,0.377194,-0.052393,-0.474840,-0.377699,-0.235976,-0.205747,...,-0.008945,1.610532,0.080877,0.031174,-0.486950,0.147848,1.040409,-0.173304,0.048496,0.744975
'56-22_H3K4me3_rep1_210115Fra_D20-5524-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.609637,1.343702,0.691313,0.884391,-0.348775,0.285336,1.139797,0.839099,0.726842,0.934604,...,0.652448,0.592498,0.712345,0.837771,-0.370354,0.370689,1.104503,0.688936,1.270353,0.542807
'72-20_H3K4me3_rep2_210115Fra_D20-5201-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.879269,0.328600,1.169870,0.548599,0.187189,0.791096,0.611769,0.882617,0.598742,0.841034,...,1.425431,1.353043,1.102290,1.230904,0.750071,0.995723,1.815377,1.074692,1.603472,1.706075
'56-22_H3K4me3_rep2_210115Fra_D20-3778-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.957026,-0.475297,-1.129206,-0.622480,-1.105340,-1.098105,-1.053575,-0.881629,-0.835838,-0.874459,...,-0.995200,-0.980190,-1.163108,-1.070293,-0.793790,-0.998026,-0.445560,-1.036477,-1.046723,-0.214667


Index,chr1:778236-779421,chr1:826712-827854,chr1:958786-959213,chr1:1012976-1013414,chr1:1013515-1013891,chr1:1019478-1020531,chr1:1024870-1025616,chr1:1069151-1069387,chr1:1115718-1116122,chr1:1144228-1144907,...,chrX:155071455-155072119,chrX:155215871-155216250,chrX:155216482-155217124,chrX:155263832-155264576,chrX:155612278-155613254,chrX:155641392-155641820,chrX:155770022-155770479,chrX:155881365-155881768,chrX:155957874-155958216,chrX:155958842-155959382
'20-20_H3K27Ac_rep2_210115Fra_D20-3776-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.811688,-1.004451,-0.972206,-1.004567,-0.930659,-0.856592,-0.921224,-1.198271,-1.070761,-0.570040,...,-0.738738,-0.736942,-0.940761,-0.877970,-0.812196,-0.698776,-0.923494,-0.936454,-0.793138,-0.540124
'72-20_H3K27Ac_rep3_210115Fra_D20-4895-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.928952,-0.781302,-0.594946,-1.127036,-1.063826,-0.607720,-0.972489,-0.626527,-0.656586,-0.799778,...,-0.768194,-0.893971,-0.874601,-0.769454,-0.928497,-0.793893,-0.412267,-0.902041,-0.873663,-0.849383
'22-20_H3K27Ac_rep2_210115Fra_D20-3773-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',-1.202501,-1.010935,-1.007877,-1.352006,-1.082553,-1.129580,-1.026409,-1.105964,-1.140912,-0.990193,...,-1.204246,-0.775439,-1.203110,-1.001884,-1.072218,-0.991178,-1.085535,-1.098056,-0.988697,-1.045377
'72-20_H3K27Ac_rep1_210115Fra_D20-5526-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.603065,0.460866,0.207480,0.080483,-0.055327,0.381555,0.005841,0.280342,0.591398,0.377229,...,0.211578,0.134609,0.202225,0.700505,0.459038,0.665819,-0.060965,0.203693,-0.216888,0.281551
'20-20_H3K27Ac_rep2-1_210115Fra_D20-5076-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.473879,1.143472,0.580110,0.900583,0.956176,1.101341,0.882266,0.601433,1.118916,0.920682,...,1.537333,1.817788,1.400145,0.814756,1.556916,1.603469,1.483534,0.969063,2.009420,1.529226
'56-22_H3K27Ac_rep1_210115Fra_D20-5522-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.082776,0.504841,1.116693,1.651083,1.400089,1.037260,1.398847,0.748069,1.052390,0.606691,...,0.728030,0.035285,0.698200,0.288826,0.795764,0.768809,1.593167,0.816959,0.494966,0.801854
'22-20_H3K27Ac_rep1_210115Fra_D20-5402-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.773851,1.528458,0.714666,1.302610,1.519239,1.560780,1.096352,1.813270,1.879609,1.576546,...,0.943834,0.889279,1.194790,2.006089,1.257737,0.583123,1.239581,1.615604,0.175401,1.002827
'22-20_H3K27Ac_rep3_210115Fra_D20-4763-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.676620,-0.449530,-0.457209,-0.256872,-0.818940,-0.635666,-0.679775,-0.647992,-0.442581,-0.755146,...,-0.564826,-0.423470,-0.676369,-0.670714,-0.779120,-0.726590,-0.658738,-0.495688,-0.508738,-0.777987
'20-20_H3K27Ac_rep1_210115Fra_D20-5406-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.823630,2.055089,2.540435,1.406558,1.788944,1.874088,2.203497,1.907555,1.351248,2.281466,...,2.196124,2.353190,2.023047,1.929739,1.894323,2.238277,1.547057,2.108468,2.049548,2.290267
'20-20_H3K27Ac_rep3_210115Fra_D20-3538-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.638368,-0.838773,-0.820776,-0.303570,-0.302841,-0.648438,-0.398126,-0.693751,-0.949637,-0.163781,...,-0.754052,-0.573519,-0.393893,-0.476091,-0.606777,-0.429055,-0.578232,-0.819589,0.195015,-0.428115


Index,chr1:788527-790364,chr1:860313-861171,chr1:861641-864435,chr1:864574-865030,chr1:865151-866637,chr1:866891-872775,chr1:872989-873634,chr1:874109-875862,chr1:882342-891444,chr1:891658-892553,...,chrY:56859503-56860565,chrY:56860859-56861167,chrY:56863278-56863932,chrY:56866321-56866861,chrY:56867506-56870396,chrY:56870614-56871622,chrY:56871910-56872550,chrY:56872818-56873204,chrY:56873557-56874690,chrY:56879321-56879994
'56-22_H3K27me3_rep3_210115Fra_D20-4893-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',-1.672761,-1.437147,-1.540973,-1.420826,-1.641582,-1.533772,-1.375699,-1.460327,-1.551953,-1.487634,...,-1.141475,-0.973501,-1.264275,-0.994174,-1.384168,-1.618440,-1.220945,-1.209040,-1.407363,-1.214323
'20-20_H3K27me3_rep3-1_210115Fra_D20-4766-2_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.751630,-0.114748,-0.576832,0.400688,-0.046630,0.144524,-0.702480,-0.270537,0.049115,0.348891,...,-0.740252,-0.613305,-0.921322,-0.566890,-0.333585,-0.293057,-0.778174,-0.286248,-0.481723,-0.240009
'20-20_H3K27me3_rep2_210115Fra_D20-5077-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.553437,0.494590,0.130082,0.323387,0.073207,0.082933,0.550277,-0.019564,0.085604,-0.051308,...,-0.388328,-0.240355,-0.378807,-0.424358,-0.597205,-0.864945,-0.841490,-0.474307,-0.571343,-0.455548
'22-20_H3K27me3_rep3_210115Fra_D20-4764-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.852981,-1.268564,-1.319177,-0.728474,-1.391599,-1.097903,-1.366061,-1.478370,-1.244316,-1.359872,...,-0.294946,-0.248160,-0.658611,-0.323272,-0.858857,-0.807743,-0.873710,-0.558993,-1.005529,-0.767979
'22-20_H3K27me3_rep1_210115Fra_D20-4169-1_NA_sequence.fastq.merged.nodup.no_chrM_MT',1.054616,0.814910,-0.216261,-0.363972,0.115915,-1.023572,0.921244,0.638783,-0.763628,-0.136915,...,-0.262107,-1.125331,0.259327,1.185478,1.275948,1.599927,2.273165,0.887822,0.744870,1.076099
'56-22_H3K27me3_rep2_210115Fra_D20-5079-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.242376,-0.196042,0.031996,-0.646428,-0.280775,-0.182403,-0.218976,-0.312891,0.036779,-0.486932,...,-0.342939,-0.445095,-0.824553,-0.463459,-0.698842,-0.649051,-0.686981,-1.326045,-0.653988,-0.418423
'56-22_H3K27me3_rep1_210115Fra_D20-5525-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.576312,-0.004028,0.256445,-0.091863,0.089658,0.091192,-0.320824,0.235681,0.070352,0.299052,...,0.219043,0.152476,0.406107,0.292954,0.219070,0.345036,0.811386,-0.066536,0.186795,-0.137331
'72-20_H3K27me3_rep3_210115Fra_D20-4896-4_NA_sequence.fastq.merged.nodup.no_chrM_MT',-1.167076,-1.402371,-1.400534,-1.258343,-1.535696,-1.281679,-1.287377,-1.361072,-1.400770,-1.457195,...,-1.165837,-0.733557,-1.104203,-0.501906,-1.037201,-0.940088,-1.019369,-0.876340,-1.208790,-1.168278
'72-20_H3K27me3_rep1_210115Fra_D20-5528-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',-0.062223,1.664746,1.286655,1.154698,1.120701,1.052195,0.058892,0.923626,1.204779,1.745005,...,0.145146,0.175813,0.027083,0.343682,0.587490,0.516129,0.926709,1.114766,0.607820,0.049163
'22-20_H3K27me3_rep1-1_210115Fra_D20-5405-3_NA_sequence.fastq.merged.nodup.no_chrM_MT',0.271619,-1.111057,-0.365396,-1.100299,-0.215436,-0.285329,-0.335517,-0.335813,-0.258748,-0.232830,...,-0.096196,0.383724,-0.052542,0.048210,-0.230934,-0.194642,0.324646,-0.310607,-0.307294,-0.095329


## PCA

In [22]:
# regular PCA with `n_components` = 80% variance explained

for assay in scaled_matrices: 
    check=PCA(n_components=.9).fit(scaled_matrices[assay])
    len(check.components_)
    len(check.components_[0])
    PCA_objects[assay] = check

4

206607

2

247764

2

72216

3

93917

4

45959

## Save Eigenvalues & Subset for Top Loadings

In [10]:
# name columns (features) same as input matrix to PCA, transpose so features are rows for the next step

for assay in PCA_objects: 
    eigenvalues[assay] = pd.DataFrame(PCA_objects[assay].components_, columns = scaled_matrices[assay].columns).T

In [11]:
# for each assay's eigenvalue by PC matrix

for assay in eigenvalues:     
    
    indices= []
    
    # for each PC 
    for column in eigenvalues[assay]: 
        # get eigenvalues for assay and PC
        # take absolute value
        # sort so highest values at top
        # take top "X" features
        # get index (feature names) 
        # make it a list and append the list of indices
        indices.extend(eigenvalues[assay][column].abs().sort_values(ascending=False).iloc[0:4000].index.to_list())
    
    assay
    
    "Length (with duplicates): " 
    len(indices) 
    "Length (without duplicates): "
    len(set(indices))
    
    # selected features per assay is de-duped list of feature names
    selected_features[assay] = list(set(indices))


'ATAC'

'Length (with duplicates): '

8000

'Length (without duplicates): '

8000

...

'H3K27me3'

'Length (with duplicates): '

8000

'Length (without duplicates): '

8000

In [12]:
for assay in selected_features: 
    pd.Series(selected_features[assay])

0       chr6:166486285-166486561
1       chr2:186503807-186504453
2        chr15:69523243-69524724
3         chr3:41710690-41711476
4         chr1:44290342-44290626
                  ...           
7995     chr19:11089201-11089716
7996    chr7:124711851-124713388
7997    chr5:177373701-177374300
7998    chr6:158691423-158691786
7999      chr8:73306527-73307111
Length: 8000, dtype: object

0         chr10:74645358-74647058
1          chr6:75374511-75378782
2            chr6:4784350-4790848
3          chr2:69585870-69587375
4       chr11:118209856-118214995
                  ...            
3995      chr21:15442879-15447054
3996      chr18:11856151-11860044
3997     chr2:219684095-219687771
3998      chr11:73594800-73597583
3999      chr19:15292742-15296309
Length: 4000, dtype: object

0          chr16:3004043-3010786
1        chr16:68233708-68237691
2       chr1:183194297-183196063
3          chr18:9333357-9336420
4       chr1:100860258-100861062
                  ...           
3995     chr22:36367650-36371511
3996    chr7:101292708-101295540
3997     chr18:49612091-49614566
3998      chr1:43529344-43533725
3999      chr2:25668925-25670579
Length: 4000, dtype: object

0         chr4:81445868-81447150
1        chr11:10314145-10314860
2         chr6:79149920-79153716
3        chr22:42994154-42995912
4       chr1:164598095-164598649
                  ...           
3995      chr7:84637245-84639182
3996      chr3:15636233-15636709
3997      chr3:65951799-65955635
3998      chr3:76242741-76243717
3999      chr7:43657871-43660409
Length: 4000, dtype: object

0          chrX:58274183-58279689
1          chr1:65252650-65254324
2         chr19:43527277-43530071
3         chr12:90339788-90341694
4        chr3:126392105-126396939
                  ...            
7995     chr9:112283611-112284753
7996      chr20:63831396-63834497
7997    chr14:100546510-100549607
7998      chr13:21059599-21060659
7999     chrX:119150324-119151959
Length: 8000, dtype: object

## Output Selected Features

In [23]:
for assay in scaled_matrices: 
    
    ######################
    # UN-SCALED MATRICES #
    ######################
    
    # transpose matrix so indices are features
    df = all_matrices[assay].T
    # filter df by selected features and write to file
    df[df.index.isin(selected_features[assay])].T.to_csv(
        "/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/2_PCA_feature_selection/output/non_scaled/{}.PCA_selected_features.non_scaled.matrix".format(assay),
         sep="\t")

#     all_matrices[assay].to_csv(
#         "/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/2_PCA_feature_selection/output/non_scaled/{}.PCA_selected_features.non_scaled.matrix".format(assay),
#         sep="\t")
    
    ###################
    # SCALED MATRICES #
    ###################
    
#     transpose matrix so indices are features
#     df = scaled_matrices[assay].T
#     # filter df by selected features and write to file
#     df[df.index.isin(selected_features[assay])].T.to_csv(
#         "/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/2_PCA_feature_selection/output/scaled/{}.PCA_selected_features.scaled.matrix".format(assay),
#          sep="\t")

    scaled_matrices[assay].to_csv(
        "/home/yraghav/MIT-Fraenkel-Lab/Projects/CHDI_NeuroLINCS/advanced_analysis/Ordinal_Regression/2_PCA_feature_selection/output/scaled/{}.PCA_selected_features.scaled.matrix".format(assay),
        sep="\t")
